In [ ]:
import ee
import os
import geemap
import geopandas as gpd
import pprint as pp

ee.Authenticate()
ee.Initialize()

project_dir = 'projects/alpod-412314/assets/' 
region = 'TUK_MRD'
region_path = f'{region}_weekly'
folder = project_dir + region_path

pp.pp(ee.data.listAssets(project_dir))
pp.pp(ee.data.listAssets(project_dir + 'region_weekly/'))

#lakes_path = f'{project_dir}Lake_extractions/{region}_extraction'
lakes_path = f'{project_dir}Lake_extractions/TUK_MRD_AND_extraction'
lake_polygons = ee.FeatureCollection(lakes_path)
weekly_mosaics = ee.data.listAssets(project_dir + 'region_weekly')
#pp.pp(weekly_mosaics)

In [16]:
img = ee.Image('projects/alpod-412314/assets/region_weekly/TUK_MRD_2019_33')

# All observed pixels (water OR land = 1) 
obs_mask = img.unmask(0)
obs_mask = obs_mask.rename('observed')
#print(obs_mask.getInfo())

# All water pixels = 2
img_wtr = ee.Image.constant(0)
img_wtr = img_wtr.where(img.select('water_occurance').eq(1), 2)
img_wtr = img_wtr.rename('wtr_occurance')

# All land pixels = 1
img_land = ee.Image.constant(0)
img_land = img_land.where(img.select('water_occurance').eq(0), 1)
img_land = img_land.rename('land_occurance')

# Lake shapes from original polygons 1=Lake, 0=Not a lake
lakes_binary = lake_polygons.reduceToImage(
    properties=['n_lakes'],
    reducer=ee.Reducer.anyNonZero()
).neq(0)
lakes_binary = lakes_binary.rename('buffered_lake')



In [17]:

expr = """
(wtr_observed == 2) ? 2 :
(land_observed == 1) ? 1 :
((buffered_lake == 1) && (land_observed == 0) && (wtr_observed != 2)) ? 3 :
0
"""
conditional = ee.Image.constant(0)
conditional = lakes_binary.expression(
    expr,
    {
        'wtr_observed': img_wtr.select('wtr_occurance'),
        'land_observed': img_land.select('land_occurance'),
        'buffered_lake': lakes_binary.select('buffered_lake'),
    }
)

conditional = conditional.rename('class')
print(conditional.getInfo())



{'type': 'Image', 'bands': [{'id': 'class', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 3}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [20]:
obs_viz = {
    'bands': ['observed'],
    'min': 0,
    'max': 1,
    'palette': ['red', 'green']
}

lakes_viz = {
    'bands': ['buffered_lake'],
    'min': 0,
    'max': 1, 
    'palette': ['white', 'blue']
}

conditional_palette = ['grey', 'brown', 'blue', 'white']


conditional_viz = {
    'band': ['class'],
    'min': 0,
    'max': 3,
    'palette': conditional_palette,
}

# Create a map to display the results
Map2 = geemap.Map(center=(65, -135), zoom=4)
#Map2.addLayer(obs_mask, obs_viz, 'Observation Mask')
Map2.addLayer(conditional, conditional_viz, 'Conditional Test')
#Map2.addLayer(lakes_binary, lakes_viz, 'Buffered Lakes')

qk_viz = {
    'bands': ['wtr_occurance'],
    'min': 0,
    'max': 2, 
    'palette': ['white', 'blue']
}
qk_viz2 = {
    'bands': ['land_occurance'],
    'min': 0,
    'max': 1, 
    'palette': ['white', 'brown']
}


#Map2.addLayer(img_wtr, qk_viz, 'water')
#Map2.addLayer(img_land, qk_viz2, 'land?')

Map2

Map(center=[65, -135], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(ch…

In [19]:
os.chdir('/Users/jmaze/Documents/projects/altimetry_lakes_v3/')

region = 'TUK_MRD'
roi = gpd.read_file(f'./data/ew_rois/{region}_weekly.shp')
polygon = roi.iloc[0].geometry

coords_list = list(polygon.exterior.coords)
gee_polygon = ee.Geometry.Polygon(coords_list, proj='EPSG:4326')

conditional = conditional.clip(gee_polygon)

In [21]:
task = ee.batch.Export.image.toDrive(
    image=conditional,
    description='conditional_test4',
    folder='temp',
    fileNamePrefix='conditional_test4',
    scale=10,
    crs='EPSG:4326',
    maxPixels=1e13
)
task.start()